In [1]:
import numpy as np
from scipy.stats import binom
from scipy.optimize import brute, differential_evolution
import matplotlib.pyplot as plt
import warnings

In [2]:
# Negative log-likelihood function
def nll_func(free_params, *args):
    dist, x = args
    nll = -dist.logpmf(x, *free_params).sum()
    if np.isnan(nll):
        nll = np.inf
    return nll

# Function to fit any discrete distributions
def fit_discrete(dist, x, bounds, optimizer=brute):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return optimizer(nll_func, bounds, args=(dist, x))

In [3]:
# Generate some samples form the binomial distribution.
n, p = 5, 0.4
rvs = binom.rvs(n, p, size=10000, random_state=123)

In [4]:
# Fitting the binomail distribution using the brute force method
fit_discrete(binom, rvs, [(0,100),(0,1)])

array([5.00000411, 0.39833459])

In [5]:
# Fitting the binomail distribution using the differential evolution method
fit_discrete(binom, rvs, [(0,100),(0,1)], optimizer=differential_evolution)

     fun: 14899.014444050032
 message: 'Optimization terminated successfully.'
    nfev: 429
     nit: 13
 success: True
       x: array([5.07169877, 0.39453257])

In [6]:
# Timing the fit method using global optimzier - brute
%timeit fit_discrete(binom, rvs, [(0,100),(0,1)])

1.77 s ± 78.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# Timing the fit method using global optimzier - differential evolution
%timeit fit_discrete(binom, rvs, [(0,100),(0,1)], optimizer=differential_evolution)

2.08 s ± 453 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
from scipy.stats import beta

# analytical MLE method for fitting the binomial distribution.
def fit_binom(x, alpha=0.5, bias_correction=True):
    s2 = x.var()
    x_bar = x.mean()
    xk = np.max(x)
    k = x.size
    # first moment estimate of n for a given 'alpha'
    n_hat_prior = xk**(alpha+1)*s2**alpha / (x_bar**alpha*(xk-x_bar)**alpha)
    if bias_correction:
        n_hat_prior = np.floor(n_hat_prior)
        n_hat_priors = np.arange(0, n_hat_prior-1)
        # final estimate for n after correction for bias
        n_hat = xk + np.sum(beta.ppf(1./k, n_hat_priors+1, n_hat_prior-n_hat_priors))
    else:
        n_hat = n_hat_prior
    p_hat = x_bar/n_hat

    return n_hat, p_hat

In [9]:
fit_binom(rvs)

(5.033577403499388, 0.3956629332083822)

In [10]:
# Timing the analytical method:
%timeit fit_binom(rvs)

857 µs ± 75.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
# For smaller values for alpha, we should get closer estimates
# even if we don't do bias correction
fit_binom(rvs, alpha=0.1, bias_correction=False)

(4.991940304939013, 0.3989631041920746)

In [12]:
# Timing analytical method without bias correction
%timeit fit_binom(rvs, alpha=0.1, bias_correction=False)

299 µs ± 9.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# Trying all the methods for different values of n, p, and k.
ns = [20, 50, 100, 200, 500, 1000]
ps = [0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9]
ks = [10, 100, 1000, 10000]
bounds = [(0, 1001), (0, 1)]

In [14]:
import sys

tot = len(ns)*len(ps)*len(ks)
results = np.empty((tot, 11))

i=0

for n in ns:
    for p in ps:
        for k in ks:
            rvs = binom.rvs(n, p, size=k, random_state=123)
            np_brute = fit_discrete(binom, rvs, bounds)
            np_de    = fit_discrete(binom, rvs, bounds, differential_evolution)
            np_anb   = fit_binom(rvs, alpha=0.1, bias_correction=False)
            np_ab    = fit_binom(rvs, bias_correction=True)
            results[i, 0]  = n
            results[i, 1]  = p
            results[i, 2]  = k
            results[i, 3]  = np_brute[0]
            results[i, 4]  = np_brute[1]
            results[i, 5]  = np_de.x[0]
            results[i, 6]  = np_de.x[1]
            results[i, 7]  = np_anb[0]
            results[i, 8]  = np_anb[1]
            results[i, 9]  = np_ab[0]
            results[i, 10] = np_ab[1]
            i+=1
            sys.stdout.write(f"\rProgress: {i/tot*100:.2f}%")
sys.stdout.write("\rDone       ")

Done       00.00%

In [15]:
import pandas as pd

results_pd = pd.DataFrame(results,
                          columns=[
                              "n", "p", "k",
                              "n_hat | Brute", "p_hat | Brute",
                              "n_hat | Differential Evolution",
                              "p_hat | Differential Evolution",
                              "n_hat | Analytical (without bias correction)",
                              "p_hat | Analytical (without bias correction)",
                              "n_hat | Analytical (with bias correction)",
                              "p_hat | Analytical (with bias correction)"
                          ])

In [16]:
results_pd

,n,p,k,n_hat | Brute,p_hat | Brute,n_hat | Differential Evolution,p_hat | Differential Evolution,n_hat | Analytical (without bias correction),p_hat | Analytical (without bias correction),n_hat | Analytical (with bias correction),p_hat | Analytical (with bias correction)
0,20.0,0.1,10.0,5.921379,0.405311,5.921403,0.405309,4.996786,0.480309,5.489016,0.437237
1,20.0,0.1,100.0,6.981749,0.283596,26.767060,0.074276,5.996826,0.330175,6.362402,0.311203
2,20.0,0.1,1000.0,18.286391,0.108387,38.325630,0.051687,7.154589,0.277025,7.496592,0.264387
3,20.0,0.1,10000.0,16.759968,0.118389,351.511961,0.005656,8.129211,0.244083,8.456971,0.234623
4,20.0,0.2,10.0,10.238007,0.429771,10.167458,0.434665,8.023918,0.548361,9.868389,0.445868
...,...,...,...,...,...,...,...,...,...,...,...
163,1000.0,0.8,10000.0,995.660538,0.803437,997.368115,0.801869,964.523085,0.829375,1608.716752,0.497260
164,1000.0,0.9,10.0,920.912070,0.974143,920.561777,0.974262,1060.902827,0.845601,1893.329142,0.473821
165,1000.0,0.9,100.0,984.574810,0.914699,990.492157,0.909238,1039.468501,0.866395,1715.333024,0.525023
166,1000.0,0.9,1000.0,998.277271,0.901850,998.454577,0.901782,1025.503597,0.877906,1623.099919,0.554677


In [18]:
results_pd.to_csv('fitting_discrete.csv')